In [1]:
import os
import sys

# Check if the notebook is running on Google Colab
if 'google.colab' in sys.modules:
    # Clone the repository
    # os.system('git clone https://github.com/orabe/gait_modulation.git')
    # Change directory to the cloned repository
    # os.chdir('gait_modulation')
    
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Change directory to the desired location in Google Drive
    os.chdir('/content/drive/MyDrive/master_thesis/gait_modulation')

In [2]:
# !rm -r logs/lstm

In [3]:
if 'google.colab' in sys.modules:    
    # Install the package
    # os.system('pip install gait_modulation')
    
    # Install the package in editable mode
    os.system('pip install -e .')

In [4]:
# %%
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.utils import plot_model

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut, cross_val_predict
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import time
import logging
import seaborn as sns
from io import StringIO
import pickle
import hashlib
import multiprocessing
from itertools import product

from gait_modulation import FeatureExtractor2
from gait_modulation import LSTMClassifier
from gait_modulation.utils.utils import load_pkl, initialize_tf, disable_xla


In [5]:
# %%
def load_data():
    """Load the preprocessed data from the pickles."""
    patient_epochs_path = os.path.join("results", "pickles", "patients_epochs.pickle")
    subjects_event_idx_dict_path = os.path.join("results", "pickles", "subjects_event_idx_dict.pickle")

    patient_epochs = load_pkl(patient_epochs_path)
    subjects_event_idx_dict = load_pkl(subjects_event_idx_dict_path)

    patient_names = np.array(list(patient_epochs.keys()))
    print(f"Loaded data for {len(patient_names)} patients.")
    return patient_epochs, subjects_event_idx_dict, patient_names


In [6]:
# %%
def preprocess_data(patient_epochs, patient_names, sfreq, feature_handling="flatten_chs", mask_vals=(0.0, 2), features_config=None, n_windows_threshold=None):

    feature_extractor = FeatureExtractor2(sfreq, features_config)

    # X_grouped is a list where each element is (n_windows_per_trial, n_features)
    X_grouped, y_grouped, groups = [], [], []
    excluded_count = 0

    for patient in patient_names:
        epochs = patient_epochs[patient]

        # Extract trial indices
        trial_indices = epochs.events[:, 1]  # Middle column contains trial index
        unique_trials = np.unique(trial_indices)
        # print(f"- Patient {patient} has {len(unique_trials)} trials")

        # Extract features and labels
        X_patient, y_patient = feature_extractor.extract_features_with_labels(epochs, feature_handling)

        # Group windows by trial
        for trial in unique_trials:
            trial_mask = trial_indices == trial  # Find windows belonging to this trial
            n_windows = sum(trial_mask)

            if n_windows_threshold is not None and n_windows > n_windows_threshold:
                # print(f"Trial {trial} has {n_windows} windows, excluding...")
                excluded_count += 1
                continue

            X_grouped.append(X_patient[trial_mask])  # Store all windows of this trial
            y_grouped.append(y_patient[trial_mask])  # Store labels for this trial
            groups.append(patient)  # Keep track of the patient

            # print(f"Trial {trial} has {n_windows} windows")
    print("Number of excluded trials:", excluded_count)

    X_padded = pad_sequences(X_grouped, dtype='float32', padding='post', value=mask_vals[0])
    y_padded = pad_sequences(y_grouped, dtype='int32', padding='post', value=mask_vals[1])

    print("Padded X shape:", X_padded.shape)
    print("Padded y shape:", y_padded.shape)

    assert not np.any(np.isnan(X_padded)), "X_grouped contains NaNs"
    assert not np.any(np.isnan(y_padded)), "y_grouped contains NaNs"
    assert X_padded.shape[0] == y_padded.shape[0] == len(groups), "X, y, and groups should have the same number of trials"
    assert X_padded.shape[1] == y_padded.shape[1], "X and y should have the same number of windows"

    padded_data_path = os.path.join("results", "padded_data.npz")
    np.savez(padded_data_path, X_padded=X_padded, y_padded=y_padded, groups=groups)
    print(f"Padded data saved at {padded_data_path}.")

    return X_padded, y_padded, groups

In [7]:
# %%
def build_pipeline(input_shape, n_windows, mask_vals):
    models = {
        'lstm': LSTMClassifier(input_shape=input_shape)
    }

    pipeline = Pipeline([
        ('scaler', 'passthrough'),
        ('classifier', models['lstm'])
    ])

    param_grid = [
        {
            'classifier__hidden_dims': [[32, 32]],
            'classifier__activations': [['tanh', 'relu']],
            'classifier__recurrent_activations': [['sigmoid', 'hard_sigmoid']],
            'classifier__dropout': [0.2],
            'classifier__dense_units': [n_windows],
            'classifier__dense_activation': ['sigmoid'],
            'classifier__optimizer': ['adam'],
            'classifier__lr': [0.001],
            'classifier__patience': [200],
            'classifier__epochs': [2],
            'classifier__batch_size': [128],
            'classifier__threshold': [0.5],
            'classifier__loss': ['binary_crossentropy'],
            'classifier__mask_vals': [mask_vals],
        }
    ]

    scoring = {
        'accuracy': make_scorer(LSTMClassifier.masked_accuracy_score),
        'f1': make_scorer(LSTMClassifier.masked_f1_score),
        'roc_auc': make_scorer(LSTMClassifier.masked_roc_auc_score),
        'precision': make_scorer(LSTMClassifier.masked_precision_score),
        'recall': make_scorer(LSTMClassifier.masked_recall_score),
    }

    if any(hasattr(model, "predict_proba") for model in models.values()):
        scoring['roc_auc'] = make_scorer(LSTMClassifier.masked_roc_auc_score,
                                        # needs_proba=True,
                                        # response_method='predict_proba',
                                        # multi_class='ovr'
        )

    return pipeline, param_grid, scoring

In [8]:
initialize_tf()

patient_epochs, subjects_event_idx_dict, patient_names = load_data()

# Slice patients for testing
patient_names = patient_names[:4]
patient_epochs = {k: patient_epochs[k] for k in patient_names}
subjects_event_idx_dict = {k: subjects_event_idx_dict[k] for k in patient_names}

sfreq = patient_epochs[patient_names[0]].info['sfreq']
feature_handling = "flatten_chs"
mask_vals = (0.0, 2)

config_path = os.path.join("configs", "features_config.json")
if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        features_config = json.load(f)
    print(f"Loaded features configuration from {config_path}.")
else:
    features_config = None
    print(f"No features configuration file found at {config_path}. Using default configuration./n")

features_config = None
if features_config is None:
    features_config = {
        'time_features': {
            # 'mean': True,
            # 'std': True,
            # 'median': True,
            # 'skew': True,
            # 'kurtosis': True,
            # 'rms': True
                # peak_to_peak = np.ptp(lfp_data, axis=2)
        },
        'freq_features': {
            'psd_raw': True,
                # psd_vals = np.abs(np.fft.rfft(lfp_data, axis=2))
            # 'psd_band_mean': True, band power!
            # 'psd_band_std': True,
            # 'spectral_entropy': True
        },
        # 'wavelet_features': {
        #     'energy': False
        # },
        # 'nonlinear_features': {
        #     'sample_entropy': True,
        #     'hurst_exponent': False
        # }
    }

X_padded, y_padded, groups = preprocess_data(patient_epochs, patient_names, sfreq, feature_handling, mask_vals, features_config)

n_features = X_padded.shape[2]
n_windows = X_padded.shape[1]
input_shape = (None, n_features)

pipeline, param_grid, scoring = build_pipeline(input_shape, n_windows, mask_vals)


Memory growth enabled for GPU PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Available devices:
  - LogicalDevice(name='/device:CPU:0', device_type='CPU')
  - LogicalDevice(name='/device:GPU:0', device_type='GPU')

Running on GPU (1 available):
  - GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
    device_name: METAL

Logical GPUs Available: 1
Logical GPU 0: LogicalDevice(name='/device:GPU:0', device_type='GPU')

TensorFlow Build Details:
Built with CUDA: False
Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow is not built with CUDA.

Using Metal API for Apple Silicon (if applicable).
Loaded data for 7 patients.
No features configuration file found at configs/features_config.json. Using default configuration./n
Number of excluded trials: 0
Padded X shape: (144, 120, 294)
Padded y shape: (144, 120)
Padded data saved at results/padded_data.npz.


In [9]:
num_cores = multiprocessing.cpu_count()
print(f"Number of available CPU cores: {num_cores}")

Number of available CPU cores: 8


In [10]:
!rm -r logs/lstm

In [11]:
# %load_ext tensorboard

In [12]:
# %tensorboard --logdir logs/lstm

In [13]:
logo = LeaveOneGroupOut()

n_splits = logo.get_n_splits(X_padded, y_padded, groups)

param_values = param_grid[0].values()
candidates = list(product(*param_values))
n_candidates = len(candidates)
total_fits = n_splits * n_candidates
print(f"Fitting {n_splits} folds for each of {n_candidates} candidates, totalling {total_fits} fits")

logging.info("Starting Grid Search...")

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=logo,
    scoring=scoring,
    refit='f1' if 'f1' in scoring else 'accuracy',
    n_jobs=num_cores,
    verbose=3,
)

Fitting 4 folds for each of 1 candidates, totalling 4 fits


In [14]:
grid_search.fit(X_padded, y_padded, groups=groups)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
Training on GPU
Training on GPU
Training on GPU
Training on GPU
Epoch 1/2
Epoch 1/2
Epoch 1/2
Epoch 1/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 72s 72s/step - MASKED_accuracy: 0.4943 - MASKED_f1_score: 0.3764 - MASKED_precision: 0.3293 - MASKED_recall: 0.4392 - MASKED_roc_auc: 0.4904 - loss: 0.6948 - learning_rate: 0.0010
1/1 ━━━━━━━━━━━━━━━━━━━━ 72s 72s/step - MASKED_accuracy: 0.5303 - MASKED_f1_score: 0.4318 - MASKED_precision: 0.3737 - MASKED_recall: 0.5114 - MASKED_roc_auc: 0.5301 - loss: 0.6913 - learning_rate: 0.0010
1/1 ━━━━━━━━━━━━━━━━━━━━ 72s 72s/step - MASKED_accuracy: 0.4447 - MASKED_f1_score: 0.3960 - MASKED_precision: 0.3408 - MASKED_recall: 0.4725 - MASKED_roc_auc: 0.4403 - loss: 0.6991 - learning_rate: 0.0010
1/1 ━━━━━━━━━━━━━━━━━━━━ 72s 72s/step - MASKED_accuracy: 0.4884 - MASKED_f1_score: 0.4133 - MASKED_precision: 0.3485 - MASKED_recall: 0.5077 - MASKED_roc_auc: 0.4731 - loss: 0.6974 - learning_rate: 0.0010
Epoch 2/2
Epoch 2

GridSearchCV(cv=LeaveOneGroupOut(),
             estimator=Pipeline(steps=[('scaler', 'passthrough'),
                                       ('classifier',
                                        LSTMClassifier(callbacks=[],
                                                       input_shape=(None,
                                                                    294)))]),
             n_jobs=8,
             param_grid=[{'classifier__activations': [['tanh', 'relu']],
                          'classifier__batch_size': [128],
                          'classifier__dense_activation': ['sigmoid'],
                          'classifier__dense_units': [120],
                          'classifier__dropout': [...
             scoring={'accuracy': make_scorer(masked_accuracy_score, response_method='predict'),
                      'f1': make_scorer(masked_f1_score, response_method='predict'),
                      'precision': make_scorer(masked_precision_score, response_method='predict'),
                      'recall': make_scorer(masked_recall_score, response_method='predict'),
                      'roc_auc': make_scorer(masked_roc_auc_score, response_method='predict')},
             verbose=3)

In [15]:
# %%
def setup_logging():
    ts = time.strftime("%Y%m%d-%H%M%S")
    model_dir = os.path.join("logs", "lstm", "models", f"run_{ts}")
    history_dir = os.path.join("logs", "lstm", "history", ts)
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(history_dir, exist_ok=True)
    
    # Set up logging
    log_stream = StringIO()
    logging.basicConfig(level=logging.INFO, stream=log_stream, format='%(asctime)s - %(levelname)s - %(message)s')
    
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    console_handler.setFormatter(formatter)
    logging.getLogger().addHandler(console_handler)
    
    return model_dir, history_dir, log_stream


model_dir, history_dir, log_stream = setup_logging()

In [16]:
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_:.4f}")
logging.info(f"Best Parameters: {grid_search.best_params_}")
logging.info(f"Best Score: {grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_.named_steps['classifier'].model
model_summary_path = os.path.join(model_dir, "best_model_summary.txt")
with open(model_summary_path, 'w') as f:
    best_model.summary(print_fn=lambda x: f.write(x + '\n'))
print(best_model.summary())
print(f"Best model summary saved at {model_summary_path}.")

best_model_path = os.path.join(model_dir, "best_lstm_model.h5")
keras_model_path = os.path.join(model_dir, 'best_lstm_model.keras')
best_model.save(best_model_path)
save_model(best_model, keras_model_path)
print(f"Best LSTM model saved at {best_model_path}.")
logging.info(f"Best LSTM model saved at {best_model_path}.")

best_params_path = os.path.join(model_dir, 'best_params.json')
cv_results_path = os.path.join(model_dir, 'cv_results.csv')
evaluation_metrics_path = os.path.join(model_dir, 'evaluation_metrics.json')

with open(best_params_path, 'w') as f:
    json.dump(grid_search.best_params_, f)
print(f"Best parameters saved at {best_params_path}")

for fold, history in enumerate(grid_search.best_estimator_.named_steps['classifier'].history_):
    history_path = os.path.join(history_dir, f'best_estimator_training_history_per_fold_and_epoch.json')
    with open(history_path, 'w') as f:
        json.dump(history, f)
    print(f"Best estimator training history per fold and epoch saved at {history_path}")


results_df = pd.DataFrame(grid_search.cv_results_)
results_df.to_csv(cv_results_path, index=False)
print(f"Cross-validation results saved at {cv_results_path}")

plot_model(best_model, to_file=os.path.join(model_dir, 'model_architecture.png'), show_shapes=True)
print(f"Model architecture plot saved at {os.path.join(model_dir, 'model_architecture.png')}.")

log_file_path = os.path.join(model_dir, 'training.log')
with open(log_file_path, 'w') as f:
    f.write(log_stream.getvalue())
print(f"Training logs saved at {log_file_path}.")

Best Parameters: {'classifier__activations': ['tanh', 'relu'], 'classifier__batch_size': 128, 'classifier__dense_activation': 'sigmoid', 'classifier__dense_units': 120, 'classifier__dropout': 0.2, 'classifier__epochs': 2, 'classifier__hidden_dims': [32, 32], 'classifier__loss': 'binary_crossentropy', 'classifier__lr': 0.001, 'classifier__mask_vals': (0.0, 2), 'classifier__optimizer': 'adam', 'classifier__patience': 200, 'classifier__recurrent_activations': ['sigmoid', 'hard_sigmoid'], 'classifier__threshold': 0.5}
Best Cross-Validation Accuracy: 0.5342


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, None, 294)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 32)       │        41,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, None, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 120)            │         3,960 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 162,410 (634.42 KB)

 Trainable params: 54,136 (211.47 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 108,274 (422.95 KB)

2025-03-22 04:39:28,691 - WARNING - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


None
Best model summary saved at logs/lstm/models/run_20250322-043928/best_model_summary.txt.
Best LSTM model saved at logs/lstm/models/run_20250322-043928/best_lstm_model.h5.
Best parameters saved at logs/lstm/models/run_20250322-043928/best_params.json
Best estimator training history per fold and epoch saved at logs/lstm/history/20250322-043928/best_estimator_training_history_per_fold_and_epoch.json
Cross-validation results saved at logs/lstm/models/run_20250322-043928/cv_results.csv
Model architecture plot saved at logs/lstm/models/run_20250322-043928/model_architecture.png.
Training logs saved at logs/lstm/models/run_20250322-043928/training.log.


In [17]:
grid_search.best_estimator_.named_steps['classifier']

LSTMClassifier(activations=['tanh', 'relu'], batch_size=128, callbacks=[],
               dense_units=120, epochs=2, hidden_dims=[32, 32],
               input_shape=(None, 294), patience=200,
               recurrent_activations=['sigmoid', 'hard_sigmoid'])

In [18]:
results_df.T

,0
mean_fit_time,141.355675
std_fit_time,0.051986
mean_score_time,6.460238
std_score_time,1.701022
param_classifier__activations,"[tanh, relu]"
param_classifier__batch_size,128
param_classifier__dense_activation,sigmoid
param_classifier__dense_units,120
param_classifier__dropout,0.2
param_classifier__epochs,2
